In [1]:
import os
import pickle
import torch
from transformers import *

To use data.metrics please install scikit-learn. See https://scikit-learn.org/stable/index.html


In [2]:
data_dir = '/home-3/msomme16@jhu.edu/scratch/shimmer/data/'
CDS_path = os.path.join(data_dir, "CDS_3600.pkl")

In [ ]:
with open(CDS_path, 'rb') as f:
    CDS = pickle.load(f)
print(CDS[10])

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
# example tokenization
from transformers import BertTokenizer, BertModel, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

masked_index = 8
tokenized_text[masked_index] = '[MASK]'
print(tokenized_text)

indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])
print(tokens_tensor)
print(tokens_tensor.shape)

In [ ]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
# segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
#     outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    outputs = model(tokens_tensor)
    predictions = outputs[0]

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
print(predicted_token)

In [66]:
# GPT-2

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')


# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()
model.to('cuda')

# original_text = "Martin Steinegger is in Peru because he "
# original_text = "The real reason Steven keeps recruiting German postdocs is "
# original_text = "The secret to giving a fun and compelling Joint Lab Meeting presentation is "
# original_text = "Donald Trump "
# original_text = "Johns Hopkins University is"
# original_text = "UC Berkeley is"
# original_text = "Why did Donald Trump "
original_text = "Finding genes is easy... The secret is "





text = original_text


for i in range(100): # not the best way to iterate, but it works
    if text[-1] != ".":
        # Encode a text inputs
        indexed_tokens = tokenizer.encode(text)

        # Convert indexed tokens in a PyTorch tensor
        tokens_tensor = torch.tensor([indexed_tokens])

        # If you have a GPU, put everything on cuda
        tokens_tensor = tokens_tensor.to('cuda')

        # Predict all tokens
        with torch.no_grad():
            outputs = model(tokens_tensor)
            predictions = outputs[0]

        # get the predicted next sub-word (in our case, the word 'man')
        predicted_index = torch.argmax(predictions[0, -1, :]).item()
        predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])

        text = predicted_text

In [67]:
print("Original text:\t\t", original_text)
print("Completed sentence:\t", predicted_text)

Original text:		 Finding genes is easy... The secret is  
Completed sentence:	 Finding genes is easy... The secret is in the genes.
